In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
from scipy.stats import sem

In [ ]:
Behavior_files, Phot_files, Pupil_files, MetaData_files, MetaDataX_files, ANIMAL_IDs, ANIMAL_VARs = DirectoryAlloc_testedit(ROOTDIR, 207, 0)

# Define loop index and subject index
loopIDX = range(3)
subjIDX = [ANIMAL_VARs[ANIMAL_IDs[i]] for i in loopIDX]
OFCIDX = list(range(len(Behavior_files)))

# Load sorted file list
file_dir = r'C:\Users\NEC_GPU\Downloads\data from Qi\DCZ'
files = natsortfiles(filename_scan(file_dir))

lumped_channel_info = []
METAMATRIX = MetaSPK_test_CNO  
# Iterate over specific files and gather channel info
for idx in subjIDX[2][-5:]:
    data = loadmat(files[idx + 10])  
    goodunit_idx = np.where((METAMATRIX['Unit_type'][idx] == 1) | 
                            (METAMATRIX['Unit_type'][idx] == 10) & 
                            (METAMATRIX['Unit_level'][idx] > 1))[0]
    
    gathered_channel_info = METAMATRIX['Unit_map'][idx][goodunit_idx, 1]  
    lumped_channel_info.append(gathered_channel_info)

# Concatenate and calculate histogram
lumped_session_channel_info = np.concatenate(lumped_channel_info)
edges_to_use = np.arange(0, 202, 2)  # Define histogram edges

mean_channel_info = []
for info in lumped_channel_info:
    hist_counts, edges = np.histogram(info, bins=edges_to_use, density=True)
    mean_channel_info.append(hist_counts)

# Convert to numpy array and calculate mean and SEM
mean_channel_info = np.array(mean_channel_info)
mean_values = np.nanmean(mean_channel_info, axis=0)
sem_values = sem(mean_channel_info, axis=0, nan_policy='omit')

# Plot the mean and SEM
plt.figure()
plt.fill_between(edges[:-1], mean_values - sem_values, mean_values + sem_values, color='gray', alpha=0.5)
plt.plot(edges[:-1], mean_values, 'k', linewidth=1.5)
plt.xlabel('Depth *10 (um)')
plt.ylabel('Fraction')
plt.show()
